<a href="https://colab.research.google.com/github/gyrusrectus/styles/blob/master/mammo_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.path.insert(0,"/content/drive/MyDrive/Capstone/Python files")

In [ ]:
import mammo_crop_flip
import config_mammo

In [ ]:
import os,time
import sklearn.metrics as metrics
import scipy.io
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import tqdm
import torch.nn as nn
from datetime import datetime
from config_mammo import Config
from mydataset_mammo import MammoTrain
import torch.optim as optim
from torch.optim import lr_scheduler


print ('*******************************************************')
start_time=time.time()
saveDir='/content/drive/MyDrive/Capstone/savedmodels'
cwd=os.getcwd()
directory=saveDir+datetime.now().strftime("%d%b_%I%M%P_")+'model'
print('Model will be saved to  :', directory)

if not os.path.exists(directory):
     os.makedirs(directory)

config  = Config()

weights = np.array([1.38, 4.27, 5.87, 1.00])

train_dir="/content/drive/MyDrive/Capstone/CSV files/train_mammo.csv"
val_dir="/content/drive/MyDrive/Capstone/CSV files/val_mammo.csv"
image_dir="/content/drive/MyDrive/Capstone/JPEG_first_part"

# make the data iterator for training data
train_data = MammoTrain(train_dir,image_dir)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=config.batchsize, shuffle=True, num_workers=2)

val_data = MammoTrain(val_dir,image_dir)
valloader = torch.utils.data.DataLoader(val_data, batch_size=config.batchsize, shuffle=True, num_workers=2)




print('----------------------------------------------------------')
#%%
# Create the object for the network

if config.gpu == True:    
    net = models.resnet18(pretrained=False)
    net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)    
    net.fc=nn.Linear(512,2)
    net.cuda()
    net.train() 
    #class_weights = torch.FloatTensor(weights).cuda(config.gpuid)
    
else:
    net = models.resnet18(pretrained=False)
    net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)    
    net.fc=nn.Linear(512,2)
    

   
# Define the optimizer
optimizer = optim.Adam(net.parameters(),lr=5e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

# Define the loss function
criterion = nn.CrossEntropyLoss()


# Iterate over the training dataset
train_loss = []
val_loss = []
acc = []


for j in range(config.epochs):  
    # Start epochs   
    runtrainloss = 0
    runvalloss = 0
    
    
    for i,data in tqdm.tqdm(enumerate(trainloader)): 
        # start iterations
        images,trainLabels = Variable(data[0]),Variable(data[1])
        
        # ckeck if gpu is available
        if config.gpu == True:
            images  = images.cuda()
            trainLabels = trainLabels.cuda()
                    
        # make forward pass      
        output = net(images)
       
        #compute loss
        loss   = criterion(output, trainLabels)        
                
        # make gradients zero
        optimizer.zero_grad()
        
        # back propagate
        loss.backward()
        
        # Accumulate loss for current minibatch
        runtrainloss += loss.item()
        
        
        # update the parameters
        optimizer.step()       
        
       
    # print loss after every epoch
    
    print('\n Training - Epoch {}/{}, loss:{:.4f} '.format(j+1, config.epochs, runtrainloss/len(trainloader)))
    train_loss.append(runtrainloss/len(trainloader))
    
       
    # Take a step for scheduler
    scheduler.step()
    
    net.eval()
    total =0
    correct = 0
    for i,data in tqdm.tqdm(enumerate(valloader)): 
         # start iterations
         images,valLabels = Variable(data[0]),Variable(data[1])
        
    #     # ckeck if gpu is available
         if config.gpu == True:
             images  = images.cuda()
             valLabels = valLabels.cuda()
                    
    #     # make forward pass      
         output = net(images)
        
    #     # Find Accuracy
         _, predicted = torch.max(F.softmax(output, dim=1), dim=1)
         total += valLabels.size(0)
         correct += (predicted == valLabels).sum().item()
       
    #     #compute loss
         loss   = criterion(output, valLabels)        
                
    #     # Accumulate loss for current minibatch
         runvalloss += loss.item()
        
           
    # # print loss after every epoch
    
    print(' \n Validatn - Epoch {}/{}, loss:{:.4f}, Acc:{:.4f}'.format(j+1, 
                 config.epochs, runvalloss/len(valloader), correct / total))
    val_loss.append(runvalloss/len(valloader))
    acc.append(correct / total)
    
    print('----------------------------------------------------------')
    
    
    #save the model   
    torch.save(net.state_dict(),os.path.join(directory,"Resnet_crop_flip" + str(j+1) +"_model.pth"))
    
    
    	    

# Save the train stats

np.save(directory + '/trnloss.npy',np.array(train_loss) )
np.save(directory + '/valloss.npy',np.array(val_loss) )
np.save(directory + '/acc.npy',    np.array(   acc    ) )


# plot the training loss

x = range(config.epochs)
plt.figure()
plt.plot(x,train_loss,label='Training')
plt.xlabel('epochs')
plt.ylabel('Train Loss ') 
plt.legend(loc="upper left")  
plt.show()
plt.figure()
plt.plot(x,val_loss,label='Validation')
plt.xlabel('epochs')
plt.ylabel('Val Loss ') 
plt.legend(loc="upper left")  
plt.show()
plt.figure()
plt.plot(x,acc,label='Validation')
plt.xlabel('epochs')
plt.ylabel('Val acc ') 
plt.legend(loc="upper left")  
plt.show()

*******************************************************
Model will be saved to  : /content/drive/MyDrive/Capstone/savedmodels14Jul_0249pm_model
----------------------------------------------------------


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
53it [24:48, 28.08s/it]


 Training - Epoch 1/50, loss:0.6921 



13it [05:08, 23.74s/it]


 
 Validatn - Epoch 1/50, loss:0.7032, Acc:0.4748
----------------------------------------------------------


...


13it [03:55, 18.10s/it]

 
 Validatn - Epoch 47/50, loss:0.6253, Acc:0.6394
----------------------------------------------------------



53it [23:09, 26.21s/it]


 Training - Epoch 48/50, loss:0.6022 



7it [02:29, 21.58s/it]